# BAYESIAN MACHINE LEARNING - PROJECT
# Practical Bayesian Optimization of Machine Learning

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
np.set_printoptions(precision=3, suppress=True, floatmode='fixed')
import numpy.random as npr
import scipy.stats as sps

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="ticks")
%matplotlib notebook

from functions import *

import warnings
warnings.filterwarnings("ignore")

## 1D example

An example with a simple target function and one parameter. 

In [ ]:
# example
def target(x, wait=True):
    if wait: time.sleep(np.abs(5*x[0]*np.sin(x[0])))
    return -x**2 * np.sin(5 * np.pi * x)**6.0   # target function

space = np.array([[0, 1]])   # parameters space
nu = 0.1   # noise standard deviation
N0 = 5   # number of random initial evaluations
N = 30   # maximum number of evaluations
seed = 1

We begin by generating some data and doing the optimization with the classical EI criterion as acquisition function. 

In [ ]:
X, y = generate_data(target, nu, N0, space, seed)

# plot
fig, ax = plt.subplots(2, figsize=(10, 8))
# Plot training data
ax[0].plot(X, y, 'ro', label="initial train set")

# Plot target function on top
xPlot = np.linspace(space[0, 0], space[0, 1], 201)
ax[0].plot(xPlot, target(xPlot, wait=False), 'b--', label="target", linewidth=2)
ax[0].legend()
ax[0].set_xlim(space[0, 0], space[0, 1])

boEI = BayesianOptimizationEI(X, y, space, kernel=1.0 * Matern(length_scale=1.0, length_scale_bounds=(1e-1, 10.0), nu=1.5) + 1.0 * WhiteKernel(noise_level=.1))
boEI.fit() # Fit the GP to the input data

# Visualize the posterior by showing samples on top of the training data above.
number_of_samples = 10
for Seed in range(number_of_samples):
    yPlot = boEI.sample_y(xPlot.reshape(-1, 1), random_state=Seed).flatten()
    ax[0].plot(xPlot, yPlot, alpha=.3, zorder=1)
plt.show()

ax[1].plot(xPlot, boEI.acquisition_criterion(xPlot.reshape(-1, 1)), label='EI')
ax[1].legend()
ax[1].set_xlim(space[0, 0], space[0, 1])

#fig.savefig('toy_EI_0.eps', format='eps', bbox_inches='tight', dpi=1200)

In [ ]:
begin = time.time()
# optimization
while boEI.X.shape[0] < N:
    # perform one step of BO
    x_next = boEI.find_next_point(number_of_restarts=10)
    y_next = eval_objective(target, x_next.reshape(1, space.shape[0]), nu=nu)
    print('Iteration {:3d}: x_next = {:1.3f}, y_next = {:1.3f}'.format(boEI.X.shape[0]+1, x_next[0], y_next[0, 0]))
    boEI.update(x_next, y_next)
    boEI.fit()

    # plot the corresponding sample
    ax[0].plot(x_next, y_next[0], marker='$'+str(boEI.X.shape[0])+'$', color='r')
    ax[1].clear()
    ax[1].plot(xPlot, boEI.acquisition_criterion(xPlot.reshape(-1, 1)), label='EI')
    ax[1].legend()
    ax[1].set_xlim(space[0, 0], space[0, 1])
    #fig.savefig('toy_EI_' + str(boEI.X.shape[0]) + '.eps', format='eps', bbox_inches='tight', dpi=1200)

end = time.time()


i_best = np.argmin(boEI.y)
x_best, y_best = boEI.X[i_best], boEI.y[i_best]
print('x_best = {:1.3f}, y_best = {:1.3f}'.format(x_best[0], y_best[0]))
print('Time: ', end-begin)

Then we process similarly with the EI per seconds criterion:

In [ ]:
X, y, t = generate_data(target, nu, N0, space, seed=1, countTime=True)

# plot
fig, ax = plt.subplots(3, figsize=(10, 8))
# Plot training data
ax[0].plot(X, y, 'ro', label="initial train set")
ax[1].plot(X, t, 'ro', label="train time")

# Plot target function on top
xPlot = np.linspace(space[0, 0], space[0, 1], 201)
ax[0].plot(xPlot, target(xPlot, wait=False), 'b--', label="target", linewidth=2)
ax[0].legend()
ax[0].set_xlim(space[0, 0], space[0, 1])
ax[1].legend()
ax[1].set_xlim(space[0, 0], space[0, 1])

boEIperS = BayesianOptimizationEIperS(X, y, t, space, kernel=1.0 * Matern(length_scale=1.0, length_scale_bounds=(1e-1, 10.0), nu=1.5) + 1.0 * WhiteKernel(noise_level=.1))
boEIperS.fit() # Fit the GP to the input data

# Visualize the posterior by showing samples on top of the training data above.
number_of_samples = 10
for seed in range(number_of_samples):
    yPlot = boEIperS.sample_y(xPlot.reshape(-1, 1), random_state=seed).flatten()
    ax[0].plot(xPlot, yPlot, alpha=.3, zorder=1)
plt.show()

ax[2].plot(xPlot, boEIperS.acquisition_criterion(xPlot.reshape(-1, 1)), label='EI/s')
ax[2].legend()
ax[2].set_xlim(space[0, 0], space[0, 1])

#fig.savefig('toy_EIperS_0.eps', format='eps', bbox_inches='tight', dpi=1200)

In [ ]:
begin = time.time()
# optimization
while boEIperS.X.shape[0] < N:
    # perform one step of BO
    x_next = boEIperS.find_next_point(number_of_restarts=10)
    y_next, t_next = eval_objective(target, x_next.reshape(1, space.shape[0]), nu=nu, countTime=True)
    print('Iteration {:3d}: x_next = {:1.3f}, y_next = {:1.3f}'.format(boEIperS.X.shape[0]+1, x_next[0], y_next[0, 0]))
    boEIperS.update(x_next, y_next, t_next)
    boEIperS.fit()

    # plot the corresponding sample
    ax[0].plot(x_next, y_next[0], marker='$'+str(boEIperS.X.shape[0])+'$', color='r')
    ax[1].plot(x_next, t_next[0], marker='$'+str(boEIperS.X.shape[0])+'$', color='r')
    ax[2].clear()
    ax[2].plot(xPlot, boEIperS.acquisition_criterion(xPlot.reshape(-1, 1)), label='EI/s')
    ax[2].legend()
    ax[2].set_xlim(space[0, 0], space[0, 1])
    #fig.savefig('toy_EIperS_' + str(boEIperS.X.shape[0]) + '.eps', format='eps', bbox_inches='tight', dpi=1200)

end = time.time()

i_best = np.argmin(boEIperS.y)
x_best, y_best = boEIperS.X[i_best], boEIperS.y[i_best]
print('x_best = {:1.3f}, y_best = {:1.3f}'.format(x_best[0], y_best[0]))
print('Time: ', end-begin)

## Application to the optimization of Machine Learning Algorithms hyperparameters

We consider different machine learning algorithms. First we define some loss (ie target functions), one with one hyperparameter, the other with two parameters:

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.linear_model import Ridge
from sklearn.kernel_ridge import KernelRidge
#from sklearn.ensemble import RandomForestClassifier
#from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error

def kernel_Ridge_loss(parameters):
    #We consider kernel Ridge regression with one parameter that we want to optimizeng='mean_squared_error').mean()
    
    R = KernelRidge(alpha=10**parameters[0], kernel='rbf', gamma=1)
    R.fit(x_train, y_train)
    
    score = mean_squared_error(y_train, R.predict(x_train))
    
    return score

def SVM_loss(parameters):
    #We consider cross valuation with two parameters that we want to optimize
    score = cross_val_score(SVC(C=10**parameters[0], gamma=10**parameters[1]), X=x_train, y=y_train).mean()    
    return score

# feel free to use another loss:
#def loss(parameters):
#    
#    return 

Then we select the Machine Learning problem we want to optimize by running one of the following cells. 

In [ ]:
## Iris dataset

from sklearn.datasets import load_iris

title = 'SVM_Iris'

x_train, y_train = load_iris(True)

# parameters
space = np.array([[-2, 5], [-6, 0]], dtype='int')
nu = 0
N0 = 5
seed = 1
N = 50
target = SVM_loss

In [ ]:
## A Sonar dataset, you can download the data at https://datahub.io/machine-learning/sonar

import pandas as pd

title = 'SVM_Sonar'

data = pd.read_csv('sonar.csv', sep=',', header=0).values

#data = np.genfromtxt('sonar.csv', delimiter=',')
x_train = data[:, 0:-1].astype('float')
y_train = np.zeros(x_train.shape[0], dtype='int')
y_train[data[:, -1] == 'Rock'] = 1

space = np.array([[-2, 5], [-6, 0]], dtype='int')
nu = 0
N0 = 5
seed = 1
N = 200
target = SVM_loss

In [ ]:
## Motorcycle data, regression, available at http://www.tsi.enst.fr/~roueff/edu/sd205/Motorcycledata.txt

title = 'SVM_Motorcycle'

data = np.loadtxt('Motorcycledata.txt')
x_train = data[:, 1].reshape(-1, 1)
y_train = data[:, -1].astype('int')

space = np.array([[-2, 5], [-6, 0]], dtype='int')
nu = 0
N0 = 5
seed = 1
N = 50
target = SVM_loss

In [ ]:
## HTRU2 dataset, available at https://archive.ics.uci.edu/ml/datasets/HTRU2. 
#Evaluations are quite long with this dataset. You can use only a small part of the data

title = 'SVM_HTRU2'

data = np.genfromtxt('HTRU_2.csv', delimiter=',')
x_train = data[:, 0:-1] 
y_train = data[:, -1].astype('int64')

space = np.array([[-2, 5], [-6, 0]], dtype='int')
nu = 0
N0 = 5
seed = 1
N = 50
target = SVM_loss

In [ ]:
## Motorcycle data, regression, available at http://www.tsi.enst.fr/~roueff/edu/sd205/Motorcycledata.txt

title = 'kernel_Ridge_Motorcycle'

data = np.loadtxt('Motorcycledata.txt')
x_train = data[:, 1].reshape(-1, 1)
y_train = data[:, -1].reshape(-1, 1)

space = np.array([[-2, 2]], dtype='int')
xPlot = np.linspace(space[0, 0], space[0, 1], 201)
nu = 0
N0 = 5
seed = 1
N = 50
target = kernel_Ridge_loss

plt.figure()
plt.scatter(x_train, y_train)

In [ ]:
## feel free to use another dataset: 

x_train = 
y_train = 

space = np.array([], dtype='int') # array of shape (dim, 2)
nu = 
N0 = 
seed = 1
N = 

### Kernel comparison

In [ ]:
# specify the kernels to compare
lst_kernels = [Matern(nu=2.5), Matern(nu=1.5), Matern(nu=0.5), RBF(), Matern(nu=2.5) + WhiteKernel()]
lst_labels = ['Matern(2,5)', 'Matern(1,5)', 'Absolute exponential', 'RBF', 'Matern(2,5) + White']
plot = True

nb_kernels = len(lst_kernels)

lst_y_best = np.zeros((nb_kernels, N))

X, y = generate_data(target, nu, N0, space, 2)
print('Initialisation with {} evaluations'.format(N0))

for (k, (kernel, label)) in enumerate(zip(lst_kernels, lst_labels)):
    print('Kernel = ' + label)
    
    boEI = BayesianOptimizationEI(X, y, space, kernel=kernel)
    boEI.fit()

    if space.shape[0] == 1 and plot:
        fig, ax = plt.subplots(2, figsize=(10, 8))
        ax[0].plot(X, y, 'ro', label="initial train set")
        ax[0].set_xlim(space[0, 0], space[0, 1])
        ax[0].legend()

    while boEI.X.shape[0] < N:
        # perform one step of BO
        x_next = boEI.find_next_point(number_of_restarts=3) # returns the maximum of your acquisition criterion
        y_next = eval_objective(target, x_next.reshape(1, space.shape[0]), nu=nu)
        if (boEI.X.shape[0]+1) % 10 == 0:
            print('Iteration {:3d}: x_next = {}, y_next = {:1.3f}'.format(boEI.X.shape[0]+1, x_next, y_next[0, 0]))
        boEI.update(x_next, y_next)
        boEI.fit()

        # plot the corresponding sample
        if space.shape[0] == 1 and plot:
            ax[0].plot(x_next, y_next[0], marker='$'+str(boEI.X.shape[0])+'$', color='r')
            ax[1].clear()
            ax[1].plot(xPlot, boEI.acquisition_criterion(xPlot.reshape(-1, 1)), label='EI')
    
    lst_y_best[k] = np.minimum.accumulate(boEI.y.reshape(N))

print(lst_y_best[:, [0, N//4, N//2, 3*N//4, -1]])

In [ ]:
plt.figure()
for (k, (kernel, label)) in enumerate(zip(lst_kernels, lst_labels)):
    plt.plot(np.arange(1, N+1), lst_y_best[k], label=label)
plt.legend()
plt.xlabel('Iteration')
plt.ylabel('Best value')
plt.title('Kernel comparison')

# plt.savefig('KernelComparison_' + title + '_.eps', format='eps', bbox_inches='tight', dpi=1200)

### Comparison between EI and EI per second

Let us execute the simple Bayesian Optimization with EI as acquisition criterion. 

In [ ]:
time0 = time.time()

X, y, t = generate_data(target, nu, N0, space, seed, countTime=True)
print('Initialisation with {} evaluations'.format(N0))

boEI = BayesianOptimizationEI(X, y, space, kernel=Matern(nu=2.5))
boEI.fit() # Fit the GP to the input data

timesEI = list(np.cumsum(t.reshape(N0)))

if space.shape[0] == 1:
    fig, ax = plt.subplots(2, figsize=(10, 8))
    xPlot = np.linspace(space[0, 0], space[0, 1], 100)
    ax[0].plot(X, y, 'ro', label="initial train set")
    ax[0].set_xlim(space[0, 0], space[0, 1])
    ax[0].legend()

while boEI.X.shape[0] < N:
    # perform one step of BO
    x_next = boEI.find_next_point(number_of_restarts=3)
    y_next = eval_objective(target, x_next.reshape(1, space.shape[0]), nu=nu)
    if (boEI.X.shape[0]+1) % 10 == 0:
        print('Iteration {:3d}: x_next = {}, y_next = {:1.3f}'.format(boEI.X.shape[0]+1, x_next, y_next[0, 0]))
    boEI.update(x_next, y_next)
    boEI.fit()
    timesEI.append(time.time()-time0)

    # plot the corresponding sample
    if space.shape[0] == 1:
        ax[0].plot(x_next, y_next[0], marker='$'+str(boEI.X.shape[0])+'$', color='r')
        ax[1].clear()
        ax[1].plot(xPlot, boEI.acquisition_criterion(xPlot.reshape(-1, 1)), label='EI')

lst_y_best_EI = np.minimum.accumulate(boEI.y.reshape(N))

#print(lst_y_best[:, [0, N//4, N//2, 3*N//4, -1]])

i_best = np.argmin(boEI.y)
x_best, y_best = boEI.X[i_best], boEI.y[i_best]
print('x_best = {}, y_best = {:1.3f}'.format(x_best, y_best[0]))

Then we do the same with EI per second. 

In [ ]:
time0 = time.time()

X, y, t = generate_data(target, nu, N0, space, seed, countTime=True)
print('Initialisation with {} evaluations'.format(N0))

boEIperS = BayesianOptimizationEIperS(X, y, t, space, kernel=Matern(nu=2.5))
boEIperS.fit() # Fit the GP to the input data

timesEIperS = list(np.cumsum(t.reshape(N0)))

if space.shape[0] == 1:
    fig, ax = plt.subplots(3, figsize=(10, 8))
    xPlot = np.linspace(space[0, 0], space[0, 1], 100)
    ax[0].plot(X, y, 'ro', label="initial train set")
    ax[0].set_xlim(space[0, 0], space[0, 1])
    ax[0].legend()
    
    ax[1].plot(X, t, 'ro', label="time")
    ax[1].set_xlim(space[0, 0], space[0, 1])
    ax[1].legend()

while boEIperS.X.shape[0] < N:
    # perform one step of BO
    x_next = boEIperS.find_next_point(number_of_restarts=3) # returns the maximum of your acquisition criterion
    y_next, t_next = eval_objective(target, x_next.reshape(1, space.shape[0]), nu=nu, countTime=True)
    if (boEIperS.X.shape[0]+1) % 10 == 0:
        print('Iteration {:3d}: x_next = {}, y_next = {:1.3f}, t_next = {:1.3f}'.format(boEIperS.X.shape[0]+1, x_next, y_next[0, 0], t_next[0, 0]))
    boEIperS.update(x_next, y_next, t_next)
    boEIperS.fit()
    timesEIperS.append(time.time()-time0)

    # plot the corresponding sample
    if space.shape[0] == 1:
        ax[0].plot(x_next, y_next[0], marker='$'+str(boEIperS.X.shape[0])+'$', color='r')
        ax[1].plot(x_next, t_next[0], marker='$'+str(boEIperS.X.shape[0])+'$', color='r')
        ax[2].clear()
        ax[2].plot(xPlot, boEIperS.acquisition_criterion(xPlot.reshape(-1, 1)), label='EI')

lst_y_best_EIperS = np.minimum.accumulate(boEIperS.y.reshape(N))

i_best = np.argmin(boEIperS.y)
x_best, y_best = boEIperS.X[i_best], boEIperS.y[i_best]
print('x_best = {}, y_best = {:1.3f}'.format(x_best, y_best[0]))

In [ ]:
plt.figure()
plt.plot(np.arange(1, N+1), lst_y_best_EI, label='EI')
plt.plot(np.arange(1, N+1), lst_y_best_EIperS, label='EI/s')
plt.legend()
plt.xlabel('Number of evaluations')
plt.ylabel('Best value')
plt.title('Optimization comparison')

#plt.savefig('Optimization_Comparison_Evaluations_' + title + '_.eps', format='eps', bbox_inches='tight', dpi=1200)

In [ ]:
plt.figure()
plt.step(timesEI, lst_y_best_EI, label='EI')
plt.step(timesEIperS, lst_y_best_EIperS, label='EI/s')
plt.legend()
plt.xlabel('Time')
plt.ylabel('Best value')
plt.title('Optimization comparison')

#plt.savefig('Optimization_Comparison_Time_' + title + '_.eps', format='eps', bbox_inches='tight', dpi=1200)